## Kiến trúc mô hình VGG_19



![Kiến trúc VGG19](https://www.mdpi.com/agriengineering/agriengineering-04-00056/article_deploy/html/images/agriengineering-04-00056-g002.png)

#### Kiến trúc chi tiết hơn
| Layer    | Input Shape     | Output Shape    | Num Kernels / Neurons | Kernel / Neuron Size | Padding | Stride |
| -------- | --------------- | --------------- | --------------------- | -------------------- | ------- | ------ |
| Conv1    | (3, 224, 224)   | (64, 224, 224)  | 64                    | 3x3                  | same    | 1      |
| Conv2    | (64, 224, 224)  | (64, 224, 224)  | 64                    | 3x3                  | same    | 1      |
| MaxPool1 | (64, 224, 224)  | (64, 112, 112)  | -                     | 2x2                  | -       | 2      |
| Conv3    | (64, 112, 112)  | (128, 112, 112) | 128                   | 3x3                  | same    | 1      |
| Conv4    | (128, 112, 112) | (128, 112, 112) | 128                   | 3x3                  | same    | 1      |
| MaxPool2 | (128, 112, 112) | (128, 56, 56)   | -                     | 2x2                  | -       | 2      |
| Conv5    | (128, 56, 56)   | (256, 56, 56)   | 256                   | 3x3                  | same    | 1      |
| Conv6    | (256, 56, 56)   | (256, 56, 56)   | 256                   | 3x3                  | same    | 1      |
| Conv7    | (256, 56, 56)   | (256, 56, 56)   | 256                   | 3x3                  | same    | 1      |
| Conv8    | (256, 56, 56)   | (256, 56, 56)   | 256                   | 3x3                  | same    | 1      |
| MaxPool3 | (256, 56, 56)   | (256, 28, 28)   | -                     | 2x2                  | -       | 2      |
| Conv9    | (256, 28, 28)   | (512, 28, 28)   | 512                   | 3x3                  | same    | 1      |
| Conv10   | (512, 28, 28)   | (512, 28, 28)   | 512                   | 3x3                  | same    | 1      |
| Conv11   | (512, 28, 28)   | (512, 28, 28)   | 512                   | 3x3                  | same    | 1      |
| Conv12   | (512, 28, 28)   | (512, 28, 28)   | 512                   | 3x3                  | same    | 1      |
| MaxPool4 | (512, 28, 28)   | (512, 14, 14)   | -                     | 2x2                  | -       | 2      |
| Conv13   | (512, 14, 14)   | (512, 14, 14)   | 512                   | 3x3                  | same    | 1      |
| Conv14   | (512, 14, 14)   | (512, 14, 14)   | 512                   | 3x3                  | same    | 1      |
| Conv15   | (512, 14, 14)   | (512, 14, 14)   | 512                   | 3x3                  | same    | 1      |
| MaxPool5 | (512, 14, 14)   | (512, 7, 7)     | -                     | 2x2                  | -       | 2      |
| AvgPool  | (512, 7, 7)     | (512, 7, 7)     | -                     | -                    | -       | -      |
| FC1      | (512, 7, 7)     | (4096,)         | 4096                  | \-                   | \-      | \-     |
| FC2      | (4096,)         | (4096,)         | 4096                  | \-                   | \-      | \-     |
| FC3      | (4096,)         | (num_classes,)  | num_classes           | \-                   | \-      | \-     |

### load mô hình vgg19 của torchvision , từ đó tự xây dựng lại mô hình VGG19

In [1]:
from torchvision.models import vgg19
import torch.nn as nn

In [2]:
# Khởi tạo mô hình VGG19
model = vgg19()
# Đếm tổng số lượng tham số
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
print(model)

Total parameters: 143667240
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size

In [ ]:
# Từ đây có thể thấy mô hình có 3 lớp chính
# - features
# - avgpool
# - classifier
# Cũng chính là tên biến lúc sau đặt

In [3]:
class VGG19(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG19, self).__init__()
        self.features = self._make_features()
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = self._make_classifier(num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _make_features(self):
        layers = []
        in_channels = 3
        # Configuration for VGG19
        cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']

        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [
                    nn.Conv2d(in_channels, v, kernel_size=3, padding=1),
                    nn.ReLU(inplace=True)
                ]
                in_channels = v

        return nn.Sequential(*layers)


    def _make_classifier(self, num_classes):
        return nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )


model = VGG19()

In [4]:
print(model)

VGG19(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), pa

In [8]:
## Kiểm tra tính hoạt động của mạng custom này
import torch

random_image = torch.rand(10, 3, 448, 448)
model(random_image).shape

torch.Size([10, 1000])

In [9]:
# Đếm tổng số lượng tham số
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 143667240


## Bài Tập
- Hãy dựng lại mô hình vgg16
```python
from torchvision.models import vgg16
import torch.nn as nn
# Khởi tạo mô hình VGG19
model = vgg16()
# Đếm tổng số lượng tham số
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
print(model)
```
- So sánh số parameter giữa việc load bằng torchvision với tự viết bằng torch.nn
- Thử tạo ngẫu nhiên và infer với mô hình
- Question: Khi thay đổi kích thước size từ 3x224x224 lên 3x448x448 thì các thông số sau thay đổi như thế nào ?
    - Số lượng parameters  
    - Tốc độ inference của mô hình

    
